This notebook showcases the interactions facilitated by the implemented Self Sovereign Identity System. Due to the involvement of three entities (Holder, Issuer, Verifier), the interaction between the three has to be simulated. Furthermore, due to the restrictions of Smart Contracts, it is impossible to showcase the deployed system as the markers would require Cryptocurrency wallets filled with that specific blockchains coin. To bypass this, this notebook will deploy the contracts within a Ganache Development Environment which simulates a test blockchain network, with multiple open and linked accounts.

You can read more about Ganache here: https://archive.trufflesuite.com/docs/ganache/ 
//Note during the creation of this project, the Ganache Project has been archived and the support stopped, however as a tool for demonstration it still up-to-date and useful for this purpose.


Below are the required imports:

In [1]:
const assert = require('assert');
const ganacheOptions = {
    logging: {
        quiet: true 
    }
};
const ganache = require('ganache');
const { Web3 } = require('web3');
const web3 = new Web3(ganache.provider(ganacheOptions));
const fs = require('fs');
const { abiRegistry, evmRegistry, abiRegistration, evmRegistration, abiCredentialIssuance, evmCredentialIssuance, abiVerification, evmVerification, abiAccessControl, evmAccessControl } = require('./compile');
const { uploadCredential, downloadCredential, uploadAttribute} = require('./contracts/SecondaryFunctions/credentialTransfer');
const { keyGeneration, createSignature, verifySignature, storePublicKeys, retrievePublicKeys} = require('./contracts/SecondaryFunctions/qrc');
const { createMerkleTree, verifyAttributeMembership } = require('./contracts/SecondaryFunctions/selectiveDisclosure');

Deployment of the five Smart Contracts into the ganache development environment: Access Control, Credential Issuance, Registration, Registry, and Verification.

In [2]:
let accounts;
async function deployContracts() {
    accounts = await web3.eth.getAccounts();
    registry = await new web3.eth.Contract(abiRegistry)
          .deploy({
            data: evmRegistry,
          })
          .send({ from: accounts[0], gas: '15000000' });
    
    registration = await new web3.eth.Contract(abiRegistration)
      .deploy({
        data: evmRegistration, 
        arguments: [registry.options.address],
      })
      .send({ from: accounts[0], gas: '15000000' });
    
    credentialIssuance = await new web3.eth.Contract(abiCredentialIssuance)
      .deploy({
        data: evmCredentialIssuance, 
        arguments: [registry.options.address],
      })
      .send({ from: accounts[0], gas: '15000000' });
    
    verification = await new web3.eth.Contract(abiVerification)
      .deploy({
        data: evmVerification, 
        arguments: [registry.options.address],
      })
      .send({ from: accounts[0], gas: '15000000' });
    
    accessControl = await new web3.eth.Contract(abiAccessControl)
      .deploy({
        data: evmAccessControl, 
        arguments: [registry.options.address],
      })
      .send({ from: accounts[0], gas: '15000000' });
}

deployContracts();

Promise { <pending> }

Example Of Three Separate Entities, Generating Private/Public Keys Off-chain, and Registering a new DID on-chain

In [3]:
let holderDID;
let issuerDID;
let verifierDID;
let holderKeyPair;
let issuerKeyPair;
async function generateEntities(){
    
    //Generate the Key Pairs off-chain
    holderKeyPair = await keyGeneration();
    issuerKeyPair = await keyGeneration();
    verifierKeyPair = await keyGeneration();

    //Create DID Identity on-chain, registering the initial DID Document public key, as each entities Public Key Hash (used for referencing)
    await registration.methods.setKeys(holderKeyPair.hash).send({ from: accounts[0], gas: '15000000' });
    let holderDidReceipt = await registration.methods.registerDID().send({ from: accounts[0], gas: '15000000' });
    holderDID = holderDidReceipt.events.DIDCreated.returnValues.did;
    
    await registration.methods.setKeys(issuerKeyPair.hash).send({ from: accounts[1], gas: '15000000' });
    let issuerDidReceipt = await registration.methods.registerDID().send({ from: accounts[1], gas: '15000000' });
    issuerDID = issuerDidReceipt.events.DIDCreated.returnValues.did;
    await registry.methods.registerPublicKey(issuerKeyPair.hash, issuerDID).send({ from: accounts[1], gas: '15000000' });
    storePublicKeys(issuerKeyPair.hash, issuerDID, issuerKeyPair.hash);
    
    await registration.methods.setKeys(verifierKeyPair.hash).send({ from: accounts[2], gas: '15000000' });
    let verifierDidReceipt = await registration.methods.registerDID().send({ from: accounts[2], gas: '15000000' });
    verifierDID = verifierDidReceipt.events.DIDCreated.returnValues.did;
    
    console.log("Holders DID:", holderDID, ", Issuers DID:", issuerDID, ", Verifiers DID:", verifierDID);
}

generateEntities();

Promise { <pending> }

Holders DID: DID:ABDNSSI:1 , Issuers DID: DID:ABDNSSI:2 , Verifiers DID: DID:ABDNSSI:3


Relevant DID Features Implemented: 

    -readDocument()      : Any user of the system can search on-chain for a specific DID identifier, and read the associated DID Document.
    
    -updateDocument()    : The owner of the DID, can modify the meta data within the associated DID document.
    
    -addVerification()   : The owner of the DID, can add new Verification Methods onto their DID Document.
    
    -deactivateDID()     : The owner of the DID can deactivate their DID and the associated DID Document.
    
    -registerPublicKey() : issuerRegistry is a on-chain mapping, which for each Issuer (Linked via DID) contains an array of public key references. This function allows Issuers to add a new public key into their array. 


Verification Method Sturcture for reference: 

    -string: id 
    
    -string: type
    
    -stirng: controller
    
    -string: public key

NOTE: When DIDs are created the sender of the transaction is recorded as the owner of the DID, and it is this account which must in the future interact with the DID document, as otherwise they will not be authorised. 

In [4]:
//function to allow for more presentable output
function formatDIDDocument(rawOutput) {
    const document = {
        id: rawOutput['0'].id,
        isActive: rawOutput['0'].isActive,
        metadata: rawOutput['0'].metadata,
        owner: rawOutput['0'].owner
    };
    const verificationMethods = rawOutput['1'].map(method => ({
        id: method.id,
        type: method._type,
        controller: method.controller,
        publicKey: method.publicKey
    }));
    const formattedDocument = {
        document,
        verificationMethods
    };

    return formattedDocument;
}



async function documentInteractions(){
    //Demonstrating Functionality of the holder interactign with their DID Document
    //Any entity can read an associated DID Document
    let didDocument = await registry.methods.readDocument(holderDID).call({from: accounts[0]});
    console.log("Output for initial DID Document");
    console.log(formatDIDDocument(didDocument));

    // The owner of the DID can update the metadata stored within the document
    await registry.methods.updateDocument(holderDID, "Example Replacement String but could be entire JSON file stringified").send({ from: accounts[0], gas: '15000000'});
    didDocument = await registry.methods.readDocument(holderDID).call({from: accounts[0]});
    console.log("Output for updated Metadata");
    console.log(formatDIDDocument(didDocument));

    // The owner can add a new Verification Method
    await registry.methods.addVerification(holderDID, "New Example Method ", holderDID, holderKeyPair.hash).send({ from: accounts[0], gas: '15000000' });
    didDocument = await registry.methods.readDocument(holderDID).call({from: accounts[0]});
    console.log("Output for added verification method");
    console.log(formatDIDDocument(didDocument));
    
    // The owner can deactivate their DID and Document
    await registry.methods.deactivateDID(holderDID).send({ from: accounts[0], gas: '15000000' });
    didDocument = await registry.methods.readDocument(holderDID).call({from: accounts[0]});
    console.log("Output for deactivated DID & Document");
    console.log(formatDIDDocument(didDocument));
    
    // An issuer can add another public key hash to their ledger
    await registry.methods.registerPublicKey(issuerKeyPair.hash, issuerDID).send({ from: accounts[1], gas: '15000000' });
    let keys = await registry.methods.retrieveKeys(issuerDID).call({from: accounts[1], gas: '15000000'});
    let success = (keys.includes(issuerKeyPair.hash));
    if (success) {console.log("Issuer Successfully registered new key");}
}
documentInteractions();

Promise { <pending> }

Output for initial DID Document
{
  document: {
    id: 'DID:ABDNSSI:1',
    isActive: true,
    metadata: 'none',
    owner: '0xE06fb86bc7Cc3999E487858F7494a08C48aF6362'
  },
  verificationMethods: [
    {
      id: 'DID:ABDNSSI:1',
      type: '2019 Public Key',
      controller: 'DID:ABDNSSI:1',
      publicKey: '28d7aee923da5994a004766d4beda902a4927cd626b8d33be0df2009bc226b41'
    }
  ]
}
Output for updated Metadata
{
  document: {
    id: 'DID:ABDNSSI:1',
    isActive: true,
    metadata: 'Example Replacement String but could be entire JSON file stringified',
    owner: '0xE06fb86bc7Cc3999E487858F7494a08C48aF6362'
  },
  verificationMethods: [
    {
      id: 'DID:ABDNSSI:1',
      type: '2019 Public Key',
      controller: 'DID:ABDNSSI:1',
      publicKey: '28d7aee923da5994a004766d4beda902a4927cd626b8d33be0df2009bc226b41'
    }
  ]
}
Output for added verification method
{
  document: {
    id: 'DID:ABDNSSI:1',
    isActive: true,
    metadata: 'Example Replacement String but coul

Interaction (Issuer <--> Holder):

    Actions & Interactions Enabled:
        - Holders can make an on-chain request for credential issuance to a specific Issuer.
        - Issuers can check on-chain, if they have any active requests for credentials. 
        - Issuers can upload/download Schemas, from on-chain storage, for use in creating Verifiable Credentials.
        - Off-chain, Issuers can fill out a schema and provide this new credential to a Holder
    


NOTE: When a new Credential is made, the system is designed so that a Merkle Tree is created allowing, for a Holder to selectively disclose specific attributes within the credential (Showcased Later). The Merkle Root is stored on-chain, allowing as a point of referenceo for verifiers to ensure the holder is providing an untampered with credential. And the Merkle Tree, is signed using the Issuers public key, allowing for later verification. Off-chain, the signature, merkle tree root, merkle tree, and an output sturcture (which contains the hash and proof for each leaf) are provided to the holder for storage.

NOTE: Signatures are made with the Quantum Resistant Cryptography Scheme: SPHINCS+, which has a large key size included in the credential, so it is advised to view the console output as a scrollable element. 

In [5]:
async function createRequest(){
    await credentialIssuance.methods.requestCredential(issuerDID, holderDID, "TestSchema").send({ from: accounts[0], gas: '15000000' }); 
    console.log("Holder Successfuly Created Request");
}
async function retrieveRequests(){
    const requests = await credentialIssuance.methods.retrieveCredentialRequest(issuerDID).send({ from: accounts[1], gas: '15000000' });
    console.log("Issuer Successfuly Retrieved Request, with this DID being the latest active request: ");
    console.log(requests.events.CredentialRequestRetrieved.returnValues.holderDID);
}

async function uploadAndDownloadSchemas(){
    const credentialData = fs.readFileSync('schemas/schemaTemplate.json', 'utf-8');
    await credentialIssuance.methods.addSchema("Example New Schema", credentialData).send({ from: accounts[1], gas: '15000000' }); 
    const schema =  await credentialIssuance.methods.getSchema("Example New Schema").call({ from: accounts[1], gas: '15000000'});
    console.log("New Schema Stored and Retrieved from On-chain");
    console.log(schema);
}

async function provisionOfCredential(){
    //Fill in a Schema of choice -> This is the test Schema
    const credential = {
        id: holderDID, 
        issuer: issuerKeyPair.hash,
        validFrom: "2023-01-01",
        validTill: "2024-02-20",
        types: ["Verifiable Credetial"], 
        credentialSubject: {
            id: holderDID,
            name: "Mark",
            age: "18",
            memberOfClub: true,
            address :"123 madeup avenue"
        }
    };
    //Issuer creates merkle tree
    const {root, tree, output} = createMerkleTree(credential);
    //Issuer signs the tree
    let message = JSON.stringify(tree);
    let privateKey = Buffer.from(issuerKeyPair.privateKeyBase64, 'base64');
    let signature = await createSignature(message, privateKey);
    //Merkle Tree Root gets sent on-chain
    await credentialIssuance.methods.issueCredential(holderDID, root, 1714924263).send({from: accounts[1], gas: '15000000'});
    //Credential Gets Saved to a File, simulating an off-chain transfer of the credential
    uploadCredential(signature, root, tree, output);
    //Holder Reads Credential From File
    const verifiableCredential = downloadCredential('testIssuer/credential.json');
    console.log("Credential Successfully exchanged, with Holder now having the below off-chain:");
    console.log(verifiableCredential);
}

createRequest();
retrieveRequests();
uploadAndDownloadSchemas();
provisionOfCredential();

Promise { <pending> }

Holder Successfuly Created Request
Issuer Successfuly Retrieved Request, with this DID being the latest active request: 
DID:ABDNSSI:1
New Schema Stored and Retrieved from On-chain
{
    "id": "Credential ID",
    "issuer": "Issuers DID",
    "validFrom": "Date of Credential Issuance, UNIX TIMESTAMP",
    "validTill": "Validity Period, left blank if no end date, UNIX TIMESTAMP",
    "signature": "digitally sign credential",
    "type": ["Type of Credential"],
    

    "credentialSubject": {
      "id": "DID of Subject", 
      "name": "",
      "age": "",
      "memberOfClub": false
    }
  }
Credential saved to file.
Credential loaded from file.
Credential Successfully exchanged, with Holder now having the below off-chain:
{
  signature: {
    '0': 96,
    '1': 116,
    '2': 91,
    '3': 9,
    '4': 242,
    '5': 145,
    '6': 3,
    '7': 136,
    '8': 15,
    '9': 101,
    '10': 242,
    '11': 153,
    '12': 211,
    '13': 194,
    '14': 194,
    '15': 103,
    '16': 162,
    '17': 

Interaction (Holder <--> Verifier): 

    Actions & Interactions Enabled:
        - Verifiers can request credentials of Holders on-chain.
        - Holders can check for Credential Requests on-chain.
        - When given a Verifiable Credential, Verifiers can check that the credential is valid (Not out of date, and the DID has not been disabled).
        - Holders can utilise selective disclosure and send a Verifier only specific attributes.
        - Verifiers can confirm these attributes are from the overall credential, and can validate the signature of the Issuer, given assurance to the overall validity of the attribute. 


In [6]:
async function verifierRequestsCredential(){
    await verification.methods.requestVerification(verifierDID, holderDID, "Request Education Proof" ).send({ from: accounts[2], gas: '15000000' });
    console.log("Verifier successfuly requested a Verifiable Credential");
}

async function holderChecksForCredential(){
    let requests = await verification.methods.getVerificationRequests(holderDID).call({ from: accounts[0], gas: '15000000' });
    console.log("Holder successfuly retrieved their requests for verification");
    console.log(requests[0].verifierDID);
}

//This function showcases Selective Disclosure, and the Validation of the Attribute/s.
async function verifierReceivesCredential(){
    const verifiableCredential = downloadCredential('testIssuer/credential.json');
    //Holder Chooses the Attribute they want to reveal from their provided Verifiable Credential
    const attr = "name";
    const attributeEntry = verifiableCredential.output.find(entry => entry[0] === attr);

    //Holder would provide this off-chain to the Verifier (Provides: attribute hash, attribute proof, merkle tree, merkle root)(All Found in the credential given to them)
    const [, hash, proof] = attributeEntry;
    uploadAttribute(verifiableCredential.signature, hash, proof, verifiableCredential.tree, verifiableCredential.root);

    //Verifier would then receive this credential- shown here as reading a file.
    const attribute = downloadCredential('testIssuer/attribute.json');
    const verifiedProof = attribute.proof.map(p => ({
          position: p.position,
          data: Buffer.isBuffer(p.data) ? p.data : Buffer.from(p.data.data)
    }));

    //Verifier could then verify that the attribute is part of the Merkle Tree first of all, to ensure the attribute belongs in the credential
    const valid = verifyAttributeMembership(Buffer.from(attribute.hash), verifiedProof, attribute.root);
    assert(valid);

    //Verifier could then verify the Issuers Signature
        //retrieves issuers key hashes from the public key registry
    const keys = await registry.methods.retrieveKeys(issuerDID).call();
        //uses hashes to get the full public key from off-chain storage
    publicKeyStorage = JSON.parse(fs.readFileSync('keyStorage.json', 'utf8'));
    const storedKeys = publicKeyStorage[issuerDID];
    let keyPair;
    for(let i = 0; i < keys.length; i++){
        keyPair = storedKeys.find(pair => pair.hash === keys[i]);
    }
    let PublicKey = Buffer.from(keyPair.publicKeyHex, 'base64');
    const verified = verifySignature(attribute.signature, PublicKey);
    assert(verified);
    
    //Finally Verifier could check the root is the same as that stored on-chain, fully ensuring the credential is valid
    const untampered = await credentialIssuance.methods.checkCredential(attribute.root, holderDID).call({ from: accounts[2], gas: '15000000' });
    assert(untampered);

    console.log("Successfully Verified Credential Attribute");
}

async function executeSequentially() {
    await verifierRequestsCredential();
    await holderChecksForCredential();
    await verifierReceivesCredential();
}
executeSequentially();

Promise { <pending> }

Verifier successfuly requested a Verifiable Credential
Holder successfuly retrieved their requests for verification
DID:ABDNSSI:3
Credential loaded from file.
Credential loaded from file.
Successfully Verified Credential Attribute


Further Auditing Provision via the Access Control Smart Contract: 

    Actions & Interactions Enabled:
        -Holders can add on-chain logs detailing Access/Consent Logs for their provided Credentials.
        -Holder can change the status of the Consent Log to revoked, allowing for clearer auditing.

In [7]:
 async function AccessControl(){
    //After providing the credential attributes to the verifier, the holder would create a consent log on-chain
    const attribute = downloadCredential('testIssuer/attribute.json');
    await accessControl.methods.logConsent(holderDID, verifierDID, attribute.root).send({ from: accounts[0], gas: '15000000' });

    //Retrieve Logs from on-chain
    let logs = await accessControl.methods.returnConsentLogs(holderDID).call({from: accounts[0]});
    console.log(logs[0]);

    //Holder can then revoke status on the log
    await accessControl.methods.revokeConsent(attribute.root, holderDID, verifierDID).send({ from: accounts[0], gas: '15000000' });
    logs = await accessControl.methods.returnConsentLogs("DID:ABDNSSI:1").call({from: accounts[0]});
    console.log(logs[0]);
 }
AccessControl();


Credential loaded from file.


Promise { <pending> }

{
  '0': '9b31581a4d0899a2ccdfd6249b7ccdc3ce75400d60ca4b3243b49c1dfa50f659',
  '1': 'DID:ABDNSSI:3',
  '2': true,
  '3': 1709757868n,
  __length__: 4,
  credentialHash: '9b31581a4d0899a2ccdfd6249b7ccdc3ce75400d60ca4b3243b49c1dfa50f659',
  verifierDID: 'DID:ABDNSSI:3',
  isConsentActive: true,
  timestamp: 1709757868n
}
{
  '0': '9b31581a4d0899a2ccdfd6249b7ccdc3ce75400d60ca4b3243b49c1dfa50f659',
  '1': 'DID:ABDNSSI:3',
  '2': false,
  '3': 1709757869n,
  __length__: 4,
  credentialHash: '9b31581a4d0899a2ccdfd6249b7ccdc3ce75400d60ca4b3243b49c1dfa50f659',
  verifierDID: 'DID:ABDNSSI:3',
  isConsentActive: false,
  timestamp: 1709757869n
}
